In [115]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/wage.csv')

In [116]:
df.shape

(753, 22)

In [117]:
cat = df.select_dtypes('object').columns
df = pd.get_dummies(df, columns = cat, drop_first = True)

In [118]:
df.columns

Index(['inlf', 'hours', 'kidslt6', 'kidsge6', 'age', 'educ', 'wage', 'repwage',
       'hushrs', 'husage', 'huseduc', 'huswage', 'faminc', 'mtr', 'motheduc',
       'fatheduc', 'unem', 'city', 'exper', 'nwifeinc', 'lwage', 'expersq'],
      dtype='object')

In [119]:
outcome = 'lwage'
treatment = 'educ'
rest = list(df.drop([outcome, treatment], axis = 1).columns)
rest = ['exper','age', 'kidslt6', 'kidsge6']
df = df[[outcome] + [treatment] + rest]

In [120]:
#import wooldridge
#df = wooldridge.data('jtrain3')
#df['avg'] = 0.5 * (df.re74+df.re75)
df = df.dropna()
#df = df.fillna(0)
#df = df[df.avg <= 15]
y = df[outcome]
d = df[treatment]
x = df[rest].astype('float')
print(df.shape)
x.head()

(428, 6)


,exper,age,kidslt6,kidsge6
0,14.0,32.0,1.0,0.0
1,5.0,30.0,0.0,2.0
2,15.0,35.0,1.0,3.0
3,6.0,34.0,0.0,3.0
4,7.0,31.0,1.0,2.0


In [122]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     14.98
Date:                Tue, 06 Dec 2022   Prob (F-statistic):           1.52e-13
Time:                        13:51:16   Log-Likelihood:                -433.15
No. Observations:                 428   AIC:                             878.3
Df Residuals:                     422   BIC:                             902.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1096      0.014      7.600      0.0

In [147]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=1)
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
est = LinearDML(discrete_treatment=False, cv=10)
est.fit(y.ravel(), d.ravel(), X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,0.109,0.014,7.935,0.0,0.082,0.136


In [152]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=1)
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
est = LinearDML(discrete_treatment=False, model_y = RandomForestRegressor(), model_t = RandomForestRegressor(), cv = 10)
est.fit(y.ravel(), d.ravel(), X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,0.116,0.014,8.434,0.0,0.089,0.143


In [54]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 8)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: lwage
Treatment variable(s): ['educ']
Covariates: ['exper', 'age', 'kidslt6', 'kidsge6']
Instrument variable(s): None
No. Observations: 428

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 428 entries, 0 to 427
Columns: 6 entries, lwage to kidsge6
dtypes: float64(1), int64(5)
memory usage: 23.4 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: lwage
Treatment variable(s): ['educ']
Covariates: ['exper', 'age', 'kidslt6', 'kidsge6']
Instrument variable(s): None
No. Observations: 428

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=8, max_features='sqrt', n

In [58]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(42)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus, n_folds = 10)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: lwage
Treatment variable(s): ['educ']
Covariates: ['exper', 'age', 'kidslt6', 'kidsge6']
Instrument variable(s): None
No. Observations: 428

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 428 entries, 0 to 427
Columns: 6 entries, lwage to kidsge6
dtypes: float64(1), int64(5)
memory usage: 23.4 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: lwage
Treatment variable(s): ['educ']
Covariates: ['exper', 'age', 'kidslt6', 'kidsge6']
Instrument variable(s): None
No. Observations: 428

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=6, max_features='sqrt')
L

In [56]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=45)
est.fit(y, d, X=None,W=x)
est.summary()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,0.112,0.014,8.006,0.0,0.084,0.139
